In [ ]:
from deep_sort_app import run
import time

In [ ]:
mot_files = [
    "testing_data/KITTI-17/",
    "testing_data/MOT16-09/",
    "testing_data/MOT16-11/",
    "testing_data/PETS09-S2L1/",
    "testing_data/TUD-Campus/",
    "testing_data/TUD-Stadtmitte/",
]
detection_models = [
    "original",
    "yolov5nano",
    "yolov5small",
    "yolov5medium",
    "yolov5large",
    "yolov5extralarge",
    "yolov10nano",
    "yolov10small",
    "yolov10medium",
    "yolov10balanced",
    "yolov10large",
    "yolov10extralarge",
    "nanodet",
]
feature_generator_models = [
    "original",
    "dpreid_shufflenet",
    "dpreid_mlfn",
    "dpreid_mobilenetv2_1_0",
    "dpreid_mobilenetv2_1_4",
    "dpreid_osnet_ibn_x1_0",
    "dpreid_osnet_ain_x1_0",
    "dpreid_osnet_ain_x0_75",
    "dpreid_osnet_ain_x0_5",
    "dpreid_osnet_ain_x0_25",
    "dpreid_osnet_x1_0",
    "dpreid_osnet_x0_75",
    "dpreid_osnet_x0_5",
    "dpreid_osnet_x0_25",
]

In [ ]:
hyperparameters = {
    "min_confidence": 0.8,
    "nms_max_overlap": 1.0,
    "min_detection_height": 0,
    "max_cosine_distance": 0.2,
    "nn_budget": None
}
display = False

In [ ]:
with open("results.txt", "a") as file:
    for d_model, fg_model in ((x, y) for x in detection_models for y in feature_generator_models):
        mean_metrics = {
            "precision": 0,
            "recall": 0,
            "f1-score": 0,
            "hota": 0,
            "fps": 0
        }
        print(d_model, fg_model)
        file.write("Detection Model: " + d_model + "\n")
        file.write("Feature Generation Model: " + fg_model + "\n")
        for seq in mot_files:
            t0 = time.time()
            metrics = run(
                seq, 
                None, 
                None, 
                hyperparameters["min_confidence"],
                hyperparameters["nms_max_overlap"],
                hyperparameters["min_detection_height"],
                hyperparameters["max_cosine_distance"],
                hyperparameters["nn_budget"],
                d_model,
                fg_model,
                display
            )
            print(seq, metrics)
            t1 = time.time()
            metrics["fps"] = metrics["frames"] / (t1 - t0)
            file.write("Video: " + seq + "\n")
            file.write("Precision: " + str(round(metrics["precision"], 2)) + "\n")
            file.write("Recall: " + str(round(metrics["recall"], 2)) + "\n")
            file.write("F1-score: " + str(round(metrics["f1-score"], 2)) + "\n")
            file.write("Hota: " + str(round(metrics["hota"], 2)) + "\n") 
            file.write("FPS: " + str(round(metrics["fps"], 2)) + "\n") 
            mean_metrics["precision"] += metrics["precision"]
            mean_metrics["recall"] += metrics["recall"]
            mean_metrics["f1-score"] += metrics["f1-score"]
            mean_metrics["hota"] += metrics["hota"]
            mean_metrics["fps"] += metrics["fps"]
        mean_metrics["precision"] = round(mean_metrics["precision"] / len(mot_files), 2)
        mean_metrics["recall"] = round(mean_metrics["recall"] / len(mot_files), 2)
        mean_metrics["f1-score"] = round(mean_metrics["f1-score"] / len(mot_files), 2)
        mean_metrics["hota"] = round(mean_metrics["hota"] / len(mot_files), 2)
        mean_metrics["fps"] = round(mean_metrics["fps"] / len(mot_files), 2)
        file.write("Mean metrics for all videos\n")
        file.write("Precision: " + str(round(mean_metrics["precision"], 2)) + "\n")
        file.write("Recall: " + str(round(mean_metrics["recall"], 2)) + "\n")
        file.write("F1-score: " + str(round(mean_metrics["f1-score"], 2)) + "\n")
        file.write("Hota: " + str(round(mean_metrics["hota"], 2)) + "\n") 
        file.write("FPS: " + str(round(mean_metrics["fps"], 2)) + "\n") 
        file.write("---------\n")

        print(mean_metrics)